In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import KFold

In [ ]:
from ipynb.fs.full.EdfManipulation import read_and_store_data
from ipynb.fs.full.FeatureSelection import dimentionalityReduction
from ipynb.fs.full.SplitDataset import createTrainingAndTestDatasets
from ipynb.fs.full.DatasetBalancing import minorityOversampling, majorityUndersampling
from ipynb.fs.full.FeatureNormalization import featureNormalization, removeNonNumericValues
from ipynb.fs.full.ClassificationPerformanceIndexes import classificationPerformanceIndexes, printClassificationPerformanceIndexes
from ipynb.fs.full.FeatureClassificationMethods import CompleteSVM, CompleteKNN, CompleteNB, CompleteDT, CompleteRF, CompleteLDA, CompleteLR, CompleteLSTM
from ipynb.fs.full.FeatureComputation import leftRightHemisphericChannels, featureExtractionLeftRight, averageChannels, featureExtractionAverage, featureExtractionFull

### Feature Extraction

In [ ]:
def featureExtraction (df, sample_rate, step, pca_tolerance, undersampling_method, undersampling_rate, undersampling_neighbors, oversampling_method, oversampling_neighbors, exp):
    if exp.upper() == 'FULL':
        ft = pd.DataFrame(featureExtractionFull (df, sample_rate, step))
    elif exp.upper() == 'AVERAGE':
        ft = pd.DataFrame(featureExtractionAverage (averageChannels(df), sample_rate, step))
    else:
        ft = pd.DataFrame(featureExtractionLeftRight (leftRightHemisphericChannels(df), sample_rate, step))
    removeNonNumericValues(ft)
    ft = featureNormalization(ft)
    print('Normalized features')
    removeNonNumericValues(ft)
    size = ft.shape
    print('Reducing features dimension')
    ft = dimentionalityReduction(ft, pca_tolerance)
    removeNonNumericValues(ft)
    print('Dimensions reduced from', size, 'to', ft.shape)
    size = ft.seizure.value_counts()
    print('Undersampling the majority class using', undersampling_method)
    ft = majorityUndersampling(ft.loc[:, ft.columns != 'seizure'], ft['seizure'], undersampling_rate, undersampling_neighbors, undersampling_method)
    removeNonNumericValues(ft)
    print('Majority class downsampled from (', size[0], ', ', ft.shape[1], ') to ', ft.shape, sep = '')
    size = ft.shape
    print('Oversampling the minority class using', oversampling_method)
    ft = minorityOversampling(ft.loc[:, ft.columns != 'seizure'], ft['seizure'], oversampling_neighbors, oversampling_method)
    ft = shuffle(ft)
    ft.reset_index(drop = True, inplace = True)
    removeNonNumericValues(ft)
    print('Minority class upsampled from (', size[0], ', ', ft.shape[1], ') to ', ft.shape, sep='')
    print('Writing features to a csv file\n')
    ft.to_csv(exp + 'Features.csv', index = False)
    return ft

### Create Training and Testing Data

In [ ]:
def trainTestData (features, test_ratio, k_fold, perfInd):
    X_train, X_test, y_train, y_test = createTrainingAndTestDatasets(features, test_ratio)
    results = pd.DataFrame(columns = perfInd)
    kf = KFold(n_splits = k_fold, shuffle = True)
    return X_train, X_test, y_train, y_test, results, kf

### Experiment

In [ ]:
def experiment (df, channels, sample_rate, time_window, test_ratio, pca_tolerance, undersampling_method, undersampling_rate, undersampling_neighbors, oversampling_method, oversampling_neighbors, k_fold, epochs, batch, dropout_percentage, loss_function, metric, step, perfInd, exp):
    if exp.upper() not in ['FULL', 'AVERAGE', 'LEFTRIGHT']:
        print('No such experiment:', exp)
        return
    else:
        print ('Executing Experiment', exp)
    ft = featureExtraction (df, sample_rate, step, pca_tolerance, undersampling_method, undersampling_rate, undersampling_neighbors, oversampling_method, oversampling_neighbors, exp)
    X_train, X_test, y_train, y_test, results, kf = trainTestData (ft, test_ratio, k_fold, perfInd)
    CompleteSVM(X_train, X_test, y_train, y_test, results, ft, kf, perfInd)
    CompleteKNN(X_train, X_test, y_train, y_test, results, exp, ft, kf, perfInd)
    CompleteNB(X_train, X_test, y_train, y_test, results, ft, kf, perfInd)
    CompleteDT(X_train, X_test, y_train, y_test, results, ft, kf, exp, perfInd)
    CompleteRF(X_train, X_test, y_train, y_test, results, exp, ft, kf, perfInd)
    CompleteLDA(X_train, X_test, y_train, y_test, results, exp, ft, kf, perfInd)
    CompleteLR(X_train, X_test, y_train, y_test, results, exp, ft, kf, perfInd)
    if exp.upper() == 'AVERAGE':
        lstm_units = 32
        dense_units = 8
    elif exp.upper() == 'LEFTRIGHT':
        lstm_units = 64
        dense_units = 16
    else:
        lstm_units = 128
        dense_units = 32  
    CompleteLSTM(X_train, X_test, y_train, y_test, results, ft, kf, perfInd, epochs, batch, lstm_units, dense_units, dropout_percentage, loss_function, metric)
    return results